## Callbacks回调函数
Callbacks回调函数的作用是：对每次训练完成时（可以是一次batch，也可以是一次迭代）的进行一次自定义操作。keras的fit是默认每次batch输出训练过程。
<br>
常用回调函数有：<br>

### `keras.callbacks.ModelCheckpoint()`保存模型<br>
```
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(filepath='model.hdf5', verbose=1, period=2)
```
`filepath`是目标文件路径，`verbose`是显示保存输出，`period`是保存模型迭代间隔。

### `keras.callbacks.EarlyStopping()`控制训练<br>
```
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='acc', patience=2, verbose=1, mode='auto')
```
`monitor`是监视目标，`patience`是进过多少个epoch之后，目标值不改变而停止的时间，mode可以为`auto`，`min`和`max`分别表示目标值不大幅度改变/下降/上升时停止。


### `keras.callbacks.LambdaCallback()`可以简单控制训练过程<br>
`LambdaCallback`可以在每个epoch前后(`on_epoch_begin`/`on_epoch_end`)，batch前后(`on_batch_begin`/`on_batch_end`)和训练前后(`on_train_begin`/`on_train_end`)调用其他函数。
<br>
```
from keras.callbacks import LambdaCallback

lambda_callback = LambdaCallback(
    # batch: batch_number logs: {'batch': batch_number, 'size': batch_size}/
    on_batch_begin=lambda batch, logs: print(batch, logs),

    # batch: batch_number logs: {'acc': acc_value, 'loss': loss_value, 'batch': batch_number, 'size': batch_size}
    on_batch_end=lambda batch, logs: print(batch, logs),

    # epoch_number {}
    on_epoch_begin=lambda epoch, logs: print(epoch, logs),

    # epoch_number {'acc': acc_value, 'loss': loss_value}
    on_epoch_end=lambda epoch, logs: print(epoch, logs),

    # nothing
    on_train_begin=lambda logs: print(logs),

    # {}
    on_train_end=lambda logs: print(logs),
)
```

### 复写`Callback`，灵活控制训练过程<br>
```
class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        pass

    def on_batch_begin(self, batch, logs=None):
        # batch training开始时
        pass

    def on_batch_end(self, batch, logs=None):
        # batch training结束时
        pass

    def on_epoch_begin(self, epoch, logs=None):
        # 每次迭代开始时
        pass

    def on_epoch_end(self, epoch, logs=None):
        # 每次迭代结束时
        pass

    def on_train_begin(self, logs=None):
        # 每次训练结束时
        pass

    def on_train_end(self, logs=None):
        # 每次训结束束时
        pass
```

In [1]:
from __future__ import print_function

# hyper-parameter
training_epoch = 100
num_classes = 10
learning_rate = 1e-3

import keras
from keras.datasets import mnist

(x_train, y_train), _ = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
image_size = x_train.shape[-1]

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(image_size,)))
model.add(Dense(10, activation='softmax'))

model.summary()

from keras.optimizers import *

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])


class LossHistory(keras.callbacks.Callback):
    def __init__(self):
        self.acc = {}
        self.loss = {}

    def on_epoch_end(self, epoch, logs=None):
        self.acc[epoch] = logs['acc']
        self.loss[epoch] = logs['loss']
        # model.save('model.h5')
        if (epoch + 1) % 10 == 0:
            print('Epoch %s, acc: %.5f, loss: %.5f' % (epoch+1, logs['acc'], logs['loss']))


history = LossHistory()

model.fit(x_train, y_train,
          batch_size=100,
          epochs=training_epoch,
          callbacks=[history],
          verbose=0)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 10, acc: 0.99293, loss: 0.02581
Epoch 20, acc: 0.99898, loss: 0.00548
Epoch 30, acc: 0.99595, loss: 0.01185
Epoch 40, acc: 1.00000, loss: 0.00025
Epoch 50, acc: 1.00000, loss: 0.00011
Epoch 60, acc: 1.00000, loss: 0.00007
Epoch 70, acc: 1.00000, loss: 0.00004
Epoch 80, acc: 0.99992, loss: 0.00028
Epoch 90, acc: 1.00000, loss: 0.00001
Epoch 100, acc: 1.00000, loss: 0.00002
